In [1]:
import numpy as np

# 定义产品数据
products = [
    {"name": "A", "dimensions": [23.5, 19.3, 7.3], "weight": 0.6, "quantity": 100},
    {"name": "B", "dimensions": [9, 9.5, 21.5], "weight": 0.3, "quantity": 60},
    {"name": "C", "dimensions": [25, 23.5, 7.5], "weight": 0.5, "quantity": 160},
]

# 定义箱体参数
box_params = {
    "max_weight": 20,  # 最大重量
    "max_length": 63.5,  # 最长边
    "max_sum": 265,  # 长+宽*2+高*2之和的最大值
    "size_step": 10,  # 外箱尺寸的步长
}

# 转换产品尺寸为整数值，以便于计算
for product in products:
    product["dimensions"] = [dim // box_params["size_step"] for dim in product["dimensions"]]  # 转换为10cm单位

# 初始化箱子列表
boxes = []

# 按照体积降序排序物品
products.sort(key=lambda x: x["dimensions"][0] * x["dimensions"][1] * x["dimensions"][2], reverse=True)

for product in products:
    for _ in range(product["quantity"]):
        # 遍历所有的箱子，尝试将物品放入能够容纳它的箱子
        for box in boxes:
            if all(np.array(box["remaining_dimensions"]) >= np.array(product["dimensions"])) and box["remaining_weight"] >= product["weight"]:
                # 更新箱子的剩余空间和重量
                box["remaining_dimensions"] = list(np.array(box["remaining_dimensions"]) - np.array(product["dimensions"]))
                box["remaining_weight"] -= product["weight"]
                box["products"].append(product["name"])
                break
        else:
            # 如果没有找到能够容纳物品的箱子，就新增一个箱子
            boxes.append({
                "dimensions": [int(box_params["max_length"] // box_params["size_step"])] * 3,
                "remaining_dimensions": [int(box_params["max_length"] // box_params["size_step"])] * 3,
                "max_weight": box_params["max_weight"],
                "remaining_weight": box_params["max_weight"] - product["weight"],
                "products": [product["name"]],
            })

# 输出箱子的数量
print("Number of boxes needed:", len(boxes))
# 输出每个箱子中物品的摆放方式
for i, box in enumerate(boxes, start=1):
    print(f"Box {i}: {box['products']}")


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Number of boxes needed: 65
Box 1: ['A', 'A', 'A', 'A', 'B', 'B', 'B']
Box 2: ['A', 'A', 'A', 'A', 'B', 'B', 'B']
Box 3: ['A', 'A', 'A', 'A', 'B', 'B', 'B']
Box 4: ['A', 'A', 'A', 'A', 'B', 'B', 'B']
Box 5: ['A', 'A', 'A', 'A', 'B', 'B', 'B']
Box 6: ['A', 'A', 'A', 'A', 'B', 'B', 'B']
Box 7: ['A', 'A', 'A', 'A', 'B', 'B', 'B']
Box 8: ['A', 'A', 'A', 'A', 'B', 'B', 'B']
Box 9: ['A', 'A', 'A', 'A', 'B', 'B', 'B']
Box 10: ['A', 'A', 'A', 'A', 'B', 'B', 'B']
Box 11: ['A', 'A', 'A', 'A', 'B', 'B', 'B'

In [7]:
import pandas as pd

# 读取excel文件
df = pd.read_excel("/Users/huzhang/Library/CloudStorage/坚果云-john.hu@39f.net/BI小组文件存档/日报表/古月日报表/20230716古月集群电商日报表.xlsx", sheet_name="订单明细")

# 筛选指定日期的数据
df = df[df['订单时间'] == '2023-07-15']

# 需要的信息映射
info_mapping = {
    "古月集群": {
        "三字机构": ["白玉京", "五丈原", "浪淘沙"]
    },
    "Amazon 美加SC": {
        "平台": ["Amazon"],
        "站点": ["CA", "US"]
    },
    "Amazon 美国VC": {
        "平台": ["Amazon_vendor"],
        "站点": ["CA", "US"]
    },
    "Amazon 欧英SC": {
        "平台": ["Amazon"],
        "站点": ["CA", "US"],
        "exclude": True
    },
    "Amazon 欧洲VC": {
        "平台": ["Amazon_vendor"],
        "站点": ["CA", "US"],
        "exclude": True
    },
    "Wayfair": {
        "平台": ["Wayfair"]
    },
    "CD": {
        "平台": ["Cdiscount"]
    },
    "Walmart": {
        "平台": ["Walmart", "Walmart_DSV", "Walmart_MP"]
    },
    "Overstock": {
        "平台": ["Overstock"]
    },
    "物权转移": {
        "平台": ["HIB"]
    }
}

# 获取各类信息的销量
for info, values in info_mapping.items():
    keys_list = list(values.keys())
    if len(keys_list) == 1:
        sum_sales = df[df[keys_list[0]].isin(values[keys_list[0]])]['销量'].sum()
    else:
        if 'exclude' in keys_list and values['exclude']:
            # 对于欧英SC和欧洲VC，排除站点为CA或US的数据
            sum_sales = df[(df[keys_list[0]].isin(values[keys_list[0]])) & (~df[keys_list[1]].isin(values[keys_list[1]]))]['销量'].sum()
        else:
            sum_sales = df[(df[keys_list[0]].isin(values[keys_list[0]])) & (df[keys_list[1]].isin(values[keys_list[1]]))]['销量'].sum()
    print(f"{info}销量：{sum_sales}")

# 古月集群需要额外分开统计水墨和大汇
shuimo_sales = df[df['三字机构'].isin(['白玉京', '五丈原'])]['销量'].sum()
dahui_sales = df[df['三字机构'] == '浪淘沙']['销量'].sum()
total_guyue_sales = shuimo_sales + dahui_sales
print(f"古月集群销量：{total_guyue_sales}（水墨{shuimo_sales}，大汇{dahui_sales}）")


古月集群销量：318
Amazon 美加SC销量：114
Amazon 美国VC销量：8
Amazon 欧英SC销量：148
Amazon 欧洲VC销量：0
Wayfair销量：11
CD销量：20
Walmart销量：5
Overstock销量：0
物权转移销量：12
古月集群销量：318（水墨236，大汇82）


In [29]:
import pandas as pd

# 读取Excel文件
df = pd.read_excel("/Users/huzhang/Library/CloudStorage/坚果云-john.hu@39f.net/BI小组文件存档/日报表/古月日报表/20230716古月集群电商日报表.xlsx", sheet_name="日报表")

# 寻找第三行对应的日期
date_row = df.iloc[1, 5:]
date_col = date_row[date_row == "2023-07-16"].index[0]

# 找到"SUM"所在的行
sum_index = df[df.iloc[:, 0] == 'SUM'].index[0]

# 选择'SUM'行以下的所有行，并在第五列中寻找对应的信息
df = df.loc[sum_index:]
df.set_index(df.columns[4], inplace=True)

# 根据日期列获取对应的数据
results = {
    '古月集群营业额': df.at['Total Amount 日总营业额 （USD)', date_col],
    '预计提现': df.at['Total Amount 日总营业额 （USD)', date_col] * 0.6,
    '古月集群毛利润': df.at['Profit 利润 (USD)', date_col],
    '古月集群净利润': df.at['Total Amount 日总营业额 （USD)', date_col] * 0.044,
    '古月集群海外库存': df.at['Oversea Stock Qty 海外在仓库存数量', date_col],
    '古月集群在途库存': df.at['On the way Stock Qty 在途库存数量', date_col],
    '古月集群库存周转天数': df.at['Turnover Days 周转天数', date_col]
}

# 输出结果
for k, v in results.items():
    print(f'{k}：{v}')


WEEK 1         2023-07-09
Unnamed: 6     2023-07-10
Unnamed: 7     2023-07-11
Unnamed: 8     2023-07-12
Unnamed: 9     2023-07-13
Unnamed: 10    2023-07-14
Unnamed: 11    2023-07-15
WEEK 2         2023-07-16
Unnamed: 13    2023-07-17
Unnamed: 14    2023-07-18
Unnamed: 15    2023-07-19
Unnamed: 16    2023-07-20
Unnamed: 17    2023-07-21
Unnamed: 18    2023-07-22
周汇总                   NaN
Unnamed: 20           NaN
Unnamed: 21           NaN
Unnamed: 22           NaN
月汇总                   NaN
Unnamed: 24           NaN
Unnamed: 25           NaN
Unnamed: 26           NaN
Unnamed: 27           NaN
Unnamed: 28           NaN
Unnamed: 29           NaN
季度汇总                  NaN
Unnamed: 31           NaN
Unnamed: 32           NaN
Unnamed: 33           NaN
Name: 1, dtype: object
古月集群营业额：22059.43
预计提现：13235.658
古月集群毛利润：1705.57
古月集群净利润：970.61492
古月集群海外库存：74097
古月集群在途库存：8880
古月集群库存周转天数：132.36


In [44]:
import pandas as pd
from datetime import date, timedelta
# 读取Excel文件
df_order = pd.read_excel("/Users/huzhang/Library/CloudStorage/坚果云-john.hu@39f.net/BI小组文件存档/日报表/古月日报表/20230716古月集群电商日报表.xlsx", sheet_name="订单明细")
df_report = pd.read_excel("/Users/huzhang/Library/CloudStorage/坚果云-john.hu@39f.net/BI小组文件存档/日报表/古月日报表/20230716古月集群电商日报表.xlsx", sheet_name="日报表")

# 筛选指定日期的订单明细数据
current_date = date.today()
previous_date = current_date - timedelta(days=1)
df_order = df_order[df_order['订单时间'] == previous_date.strftime("%Y-%m-%d")]

# 需要的信息映射
info_mapping = {
    "古月集群": {
        "三字机构": ["白玉京", "五丈原", "浪淘沙"]
    },
    "Amazon 美加SC": {
        "平台": ["Amazon"],
        "站点": ["CA", "US"]
    },
    "Amazon 美国VC": {
        "平台": ["Amazon_vendor"],
        "站点": ["CA", "US"]
    },
    "Amazon 欧英SC": {
        "平台": ["Amazon"],
        "站点": ["CA", "US"],
        "exclude": True
    },
    "Amazon 欧洲VC": {
        "平台": ["Amazon_vendor"],
        "站点": ["CA", "US"],
        "exclude": True
    },
    "Wayfair": {
        "平台": ["Wayfair"]
    },
    "CD": {
        "平台": ["Cdiscount"]
    },
    "Walmart": {
        "平台": ["Walmart", "Walmart_DSV", "Walmart_MP"]
    },
    "Overstock": {
        "平台": ["Overstock"]
    },
    "物权转移": {
        "平台": ["HIB"]
    }
}

# 构建输出结果的字符串变量
output_text = f"日期: {current_date.strftime('%m月%d日')}\n"

# 古月集群单量合并
shuimo_sales = df_order[df_order['三字机构'].isin(['白玉京', '五丈原'])]['销量'].sum()
dahui_sales = df_order[df_order['三字机构'] == '浪淘沙']['销量'].sum()
total_guyue_sales = shuimo_sales + dahui_sales
output_text += f"古月集群单量: {total_guyue_sales}(水墨{shuimo_sales},大汇{dahui_sales})\n"

for info, values in info_mapping.items():

    keys_list = list(values.keys())
    if len(keys_list) == 1:
        sum_sales = df_order[df_order[keys_list[0]].isin(values[keys_list[0]])]['销量'].sum()
    else:
        if 'exclude' in keys_list and values['exclude']:
            sum_sales = df_order[(df_order[keys_list[0]].isin(values[keys_list[0]])) & (~df_order[keys_list[1]].isin(values[keys_list[1]]))]['销量'].sum()
        else:
            sum_sales = df_order[(df_order[keys_list[0]].isin(values[keys_list[0]])) & (df_order[keys_list[1]].isin(values[keys_list[1]]))]['销量'].sum()
    if info != "古月集群":
        output_text += f"{info}单量: {sum_sales}\n"
# 寻找日期对应的列索引
date_row = df_report.iloc[1, 5:]
date_col = date_row[date_row == current_date.strftime("%Y-%m-%d")].index[0]

# 找到"SUM"所在的行
sum_index = df_report[df_report.iloc[:, 0] == 'SUM'].index[0]

# 选择'SUM'行以下的所有行，并在第五列中寻找对应的信息
df_report = df_report.loc[sum_index:]
df_report.set_index(df_report.columns[4], inplace=True)

# 根据日期列获取对应的数据
results = {
    '古月集群营业额': int(df_report.at['Total Amount 日总营业额 （USD)', date_col]),
    '预计提现': int(df_report.at['Total Amount 日总营业额 （USD)', date_col] * 0.6),
    '古月集群毛利润': int(df_report.at['Profit 利润 (USD)', date_col]),
    '古月集群净利润': int(df_report.at['Total Amount 日总营业额 （USD)', date_col] * 0.044),
    '古月集群海外库存': int(df_report.at['Oversea Stock Qty 海外在仓库存数量', date_col]),
    '古月集群在途库存': int(df_report.at['On the way Stock Qty 在途库存数量', date_col]),
    '古月集群库存周转天数': int(df_report.at['Turnover Days 周转天数', date_col])
}

for k, v in results.items():
    output_text += f'{k}: {v}\n'

print(output_text)


日期: 07月15日
古月集群单量: 318(水墨236,大汇82)
Amazon 美加SC单量: 114
Amazon 美国VC单量: 8
Amazon 欧英SC单量: 148
Amazon 欧洲VC单量: 0
Wayfair单量: 11
CD单量: 20
Walmart单量: 5
Overstock单量: 0
物权转移单量: 12
古月集群营业额: 22059
预计提现: 13235
古月集群毛利润: 1705
古月集群净利润: 970
古月集群海外库存: 74097
古月集群在途库存: 8880
古月集群库存周转天数: 132

